## Origin-Destination Analysis

In [26]:
import pandas as pd
def fahrenheit_to_celsius(f):
        return (f - 32) * 5/9

In [12]:
pwd()

'c:\\Users\\max-admin\\Desktop\\Masterstudium\\Masterarbeit\\Master_Thesis\\Data'

#### Cutoff: 25 degrees - Hot days 

#### Pool Origin Destination Data - Only by trip count

In [43]:
pooled_trips_OD = pd.read_csv('Pooled_data/pooled_trips_OD.csv')

In [22]:
## read in climate data and merge by date

climate_NYC = pd.read_csv("NYC_weather\climate_data_NYC_2014_2019.csv")

## Origin

In [44]:
Pooled_trips_Origin_Park = pooled_trips_OD[pooled_trips_OD['PULocationID'] == 93]


# calculate the average daily trips from each zone to the park

average_daily_trips_origin = Pooled_trips_Origin_Park.groupby('DOLocationID')['trip_count'].mean().reset_index()

# Renaming the column for clarity
average_daily_trips_origin.rename(columns={'trip_count': 'average_daily_trips'}, inplace=True)


# merge trips and climate data on date
Pooled_origin_Park = pd.merge(Pooled_trips_Origin_Park, climate_NYC, how='left', left_on='date_pickup', right_on='DATE')
# Apply the conversion function to the Fahrenheit column
Pooled_origin_Park['TMAX'] = Pooled_origin_Park['TMAX'].apply(fahrenheit_to_celsius)
# get average daily trips to park when the temperature is above 25 degrees
Pooled_origin_Park_above25 = Pooled_origin_Park[Pooled_origin_Park['TMAX'] > 25]

average_daily_trips_origin_above25 = Pooled_origin_Park_above25.groupby('DOLocationID')['trip_count'].mean().reset_index()

average_daily_trips_origin_above25.rename(columns={'trip_count': 'average_daily_trips_25'}, inplace=True)

# merge the 2 dataframes on location ID and calculate the percentage change
avg_origin = pd.merge(average_daily_trips_origin, average_daily_trips_origin_above25, how='left', on='DOLocationID')
avg_origin["deviation"] = (avg_origin["average_daily_trips_25"] - avg_origin["average_daily_trips"]) / avg_origin["average_daily_trips_25"]

### Destination

In [52]:
Pooled_trips_Destination_Park = pooled_trips_OD[pooled_trips_OD['DOLocationID'] == 93]

# calculate the average daily trips from each zone to the park

average_daily_trips_destination = Pooled_trips_Destination_Park.groupby('PULocationID')['trip_count'].mean().reset_index()

# Renaming the column for clarity
average_daily_trips_destination.rename(columns={'trip_count': 'average_daily_trips'}, inplace=True)


# merge trips and climate data on date
Pooled_destination_Park = pd.merge(Pooled_trips_Destination_Park, climate_NYC, how='left', left_on='date_pickup', right_on='DATE')
# Apply the conversion function to the Fahrenheit column
Pooled_destination_Park['TMAX'] = Pooled_destination_Park['TMAX'].apply(fahrenheit_to_celsius)
# get average daily trips to park when the temperature is above 25 degrees
Pooled_destination_Park_above25 = Pooled_destination_Park[Pooled_origin_Park['TMAX'] > 25]

average_daily_trips_destination_above25 = Pooled_destination_Park_above25.groupby('PULocationID')['trip_count'].mean().reset_index()

average_daily_trips_destination_above25.rename(columns={'trip_count': 'average_daily_trips_25'}, inplace=True)

# merge the 2 dataframes on location ID and calculate the percentage change
avg_destination = pd.merge(average_daily_trips_destination, average_daily_trips_destination_above25, how='left', on='PULocationID')
avg_destination["deviation"] = (avg_destination["average_daily_trips_25"] - avg_destination["average_daily_trips"]) / avg_destination["average_daily_trips_25"]

C:\Users\max-admin\AppData\Local\Temp\ipykernel_18244\3808391053.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  Pooled_destination_Park_above25 = Pooled_destination_Park[Pooled_origin_Park['TMAX'] > 25]


In [60]:
avg_destination["average_daily_trips_25"].describe()

count    255.000000
mean       2.417370
std        1.620091
min        1.000000
25%        1.393939
50%        1.882353
75%        2.826524
max       11.229025
Name: average_daily_trips_25, dtype: float64

In [62]:
avg_destination.to_csv('Pooled_data/avg_destination_coronapark.csv')